In [1]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, roc_auc_score, roc_curve
import pandas as pd
import numpy as np
import pygal
from pandas.io.json import json_normalize

In [44]:
# df = pd.read_csv('/home/angeles/Documentos/Diplomado_DS/Modulo_2/OnlineNewsPopularity.csv')
path_datos = "/home/lalo/Documentos/Diplomado Ciencia de Datos/Modulo 2 Modelaciòn Supervisada/Bases de Datos/"
df = pd.read_csv(path_datos + "OnlineNewsPopularity.csv")
df_aux = pd.read_csv(path_datos + "OnlineNewsPopularity.csv")

df.columns =[x.strip() for x in df.columns]
df_aux.columns =[x.strip() for x in df_aux.columns]

In [3]:
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.700000,-0.350000,-0.600000,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.700000,-0.118750,-0.125000,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.000000,-0.466667,-0.800000,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.800000,-0.369697,-0.600000,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.000000,-0.220192,-0.500000,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.600000,-0.195000,-0.400000,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.000000,-0.224479,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.000000,-0.242778,-0.500000,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.800000,-0.125000,-0.125000,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.500000,-0.238095,-0.500000,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [4]:
df.dtypes

url                               object
 timedelta                       float64
 n_tokens_title                  float64
 n_tokens_content                float64
 n_unique_tokens                 float64
                                  ...   
 title_subjectivity              float64
 title_sentiment_polarity        float64
 abs_title_subjectivity          float64
 abs_title_sentiment_polarity    float64
 shares                            int64
Length: 61, dtype: object

In [34]:
var_cont = ['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']

In [35]:
var_disc = ['data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 
           'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'weekday_is_monday',
           'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
           'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend']

In [7]:
df.columns = [x.strip() for x in df.columns]

In [8]:
df[['shares']]

,shares
0,593
1,711
2,1500
3,1200
4,505
5,855
6,556
7,891
8,3600
9,710


# Quitamos los outlyers

In [42]:
def New_poblacion_Met_2(data_origen,var, enes=3):
    data = pd.DataFrame(data_origen)
    
    sigma = data[var].std()
    media = data[var].mean()
    lim_inf  = media - enes*sigma
    lim_sup = media + enes*sigma
#     data = data[(lim_inf <= data[var])  & (data[var] <= lim_sup)]
#     Nuevos_registros, Nuevas_columnas = data.shape
    
    return lim_inf,lim_sup

In [45]:
for var_cont_i in var_cont:
    lim_abajo,lim_arriba = New_poblacion_Met_2(df_aux,var_cont_i,5)
    df = df[(lim_abajo <= df[var_cont_i]) & (df[var_cont_i] <= lim_arriba)]
  

df.shape

(35981, 61)

## Termina procedimiento para quitar outlyers 

#### Usando Regresión Lineal, estime la variable objetivo

In [53]:
# Se retoma la tabla a 35 mil
df_final =  pd.DataFrame(df)
df_final.shape

(35981, 62)

In [54]:
mr = LinearRegression()

In [55]:
mr.fit(X = df_final[var_cont], y = df_final['shares'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [58]:
mr.score(df_final[var_cont], df_final['shares'])

0.023044206330734562

In [59]:
df_final["y_hat"] = mr.predict(X = df_final[var_cont])

#### Ajuste la Regresión Lineal 

In [60]:
r2_score(df['shares'], df['y_hat'])

0.023044206330734562

In [61]:
r2_score(df_final['shares'], df_final['y_hat'])

0.023044206330734562

In [62]:
pema = (abs(df['shares'] - df['y_hat'])/df['shares'])*(1/df.shape[0])

In [63]:
sum(pema)

1.5891087237822992

In [64]:
pema = (abs(df_final['shares'] - df_final['y_hat'])/df_final['shares'])*(1/df_final.shape[0])

In [65]:
sum(pema)

1.5891087237822992

In [66]:
kb = SelectKBest(k=10, score_func=f_regression)
kb.fit(X=df[var_cont], y=df["shares"])

/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:293: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/lalo/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(k=10, score_func=<function f_regression at 0x7f20542b9a60>)

In [67]:
k_best = [x for x, y in zip([x for x in df.columns if x != "shares"], kb.get_support()) if y]

In [68]:
k_best

['n_non_stop_unique_tokens',
 'num_self_hrefs',
 'kw_min_min',
 'kw_max_min',
 'kw_avg_min',
 'kw_min_max',
 'kw_max_max',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_max_shares']

In [69]:
[(x,y) for x,y in zip(k_best,mr.coef_)]

[('n_non_stop_unique_tokens', 1.6188226118400413),
 ('num_self_hrefs', 101.63928207583471),
 ('kw_min_min', 0.6417256285421065),
 ('kw_max_min', 455.16776241621864),
 ('kw_avg_min', -61907381487.21757),
 ('kw_min_max', 263.4183056749449),
 ('kw_max_max', 33.91569786507585),
 ('kw_max_avg', -60.49461347824865),
 ('kw_avg_avg', 29.931464770136227),
 ('self_reference_max_shares', 72.63311698521655)]

In [70]:
mr.fit(X = df[k_best], y = df['shares'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [71]:
mr.score(df[k_best], df['shares'])

0.01811919956163477

#### Construya una variable objetivo discreta en función de la original. 

In [74]:
df["tgt"] = (df["shares"] > df["shares"].mean()).astype(int)

In [75]:
df["tgt"].mean()

0.20485811956310274

### Regresión logistica

In [76]:
lr = LogisticRegression(n_jobs=-1)

In [77]:
lr.fit(X=df[var_cont], y=df['tgt'])

/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
lr.score(df[var_cont], df['tgt'])

0.7957811066952002

In [79]:
df["y_hat3"] = lr.predict(X = df[var_cont])

In [80]:
df[['tgt', 'y_hat3']]

,tgt,y_hat3
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,1,0
9,0,0


In [81]:
df['y2'] = (df['tgt'] == df['y_hat3']) * 1

In [82]:
df.y2.sum()

28633

In [83]:
roc_auc_score(y_true=df['tgt'], y_score=lr.predict(X=df[var_cont]))

0.5168183375045778

#### Modele mediante Regresión Logistica la nueva variable

In [84]:
df.shape

(35981, 65)

In [85]:
for var in var_cont:
    df[f"C_{var}"] = pd.cut(df[var], bins=5).astype(str)

In [86]:
df.columns

Index(['url', 'timedelta', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs',
       ...
       'C_avg_positive_polarity', 'C_min_positive_polarity',
       'C_max_positive_polarity', 'C_avg_negative_polarity',
       'C_min_negative_polarity', 'C_max_negative_polarity',
       'C_title_subjectivity', 'C_title_sentiment_polarity',
       'C_abs_title_subjectivity', 'C_abs_title_sentiment_polarity'],
      dtype='object', length=110)

In [87]:
def IV(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux["IV"] = (aux["%no_evento"] - aux["%evento"])*aux["WOE"]
    return aux["IV"].sum()

In [88]:
new_disc = [x for x in df.columns if x.startswith("C_")] + var_disc

In [89]:
df_iv = pd.DataFrame(columns=["iv"])
for var in new_disc:
    df_iv.loc[var, "iv"] = IV(df = df, var = var, tgt = 'tgt')

In [90]:
df_iv.sort_values(by = "iv", ascending=False)

,iv
C_kw_min_min,inf
C_n_non_stop_words,inf
C_kw_max_max,inf
C_kw_avg_avg,0.19476
C_kw_min_avg,0.137976
C_kw_max_avg,0.118442
C_LDA_02,0.118191
data_channel_is_world,0.0980421
C_LDA_03,0.0788035
C_global_subjectivity,0.0608649


In [91]:
ls_best = df_iv[(df_iv["iv"] != np.inf)].index.tolist()

In [92]:
def WOE(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux.columns = aux.columns.droplevel(1)
    aux = aux[["WOE"]].reset_index().rename(columns={"WOE": f"W_{var}"})
    df = df.merge(aux, on = var, how = "left")
    display(aux)
    return df

In [93]:
for col in ls_best:
    df = WOE(df = df, var = col, tgt = 'tgt')

,C_timedelta,W_C_timedelta
0,"(152.6, 297.2]",0.000852
1,"(297.2, 441.8]",-0.252249
2,"(441.8, 586.4]",0.020782
3,"(586.4, 731.0]",-0.039448
4,"(7.277, 152.6]",0.254622


,C_n_tokens_title,W_C_n_tokens_title
0,"(13.2, 16.6]",0.020940
1,"(16.6, 20.0]",0.030091
2,"(2.983, 6.4]",-0.182083
3,"(6.4, 9.8]",-0.069709
4,"(9.8, 13.2]",0.046151


,C_n_tokens_content,W_C_n_tokens_content
0,"(1170.8, 1747.2]",-0.233674
1,"(15.118, 594.4]",0.000135
2,"(1747.2, 2323.6]",-0.294060
3,"(2323.6, 2900.0]",-0.540453
4,"(594.4, 1170.8]",0.082540


,C_n_unique_tokens,W_C_n_unique_tokens
0,"(0.181, 0.341]",-0.379480
1,"(0.341, 0.501]",-0.008743
2,"(0.501, 0.66]",0.051818
3,"(0.66, 0.82]",-0.163002
4,"(0.82, 0.98]",0.106375


,C_n_non_stop_unique_tokens,W_C_n_non_stop_unique_tokens
0,"(0.219, 0.376]",-0.193052
1,"(0.376, 0.532]",-0.312618
2,"(0.532, 0.688]",-0.018675
3,"(0.688, 0.844]",0.073882
4,"(0.844, 1.0]",-0.166539


...

,weekday_is_thursday,W_weekday_is_thursday
0,0.0,-0.019774
1,1.0,0.091138


,weekday_is_friday,W_weekday_is_friday
0,0.0,-0.003816
1,1.0,0.022868


,weekday_is_saturday,W_weekday_is_saturday
0,0.0,0.034017
1,1.0,-0.458378


,weekday_is_sunday,W_weekday_is_sunday
0,0.0,0.039697
1,1.0,-0.474104


,is_weekend,W_is_weekend
0,0.0,0.080162
1,1.0,-0.466677


In [94]:
ls_woe = [x for x in df.columns if x.startswith("W")]

In [95]:
lr = LogisticRegression(n_jobs=-1) 

In [96]:
lr.fit(X=df[ls_woe], y=df['tgt'])

/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lalo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### Ajuste de la Regresión Logistica

In [97]:
lr.score(df[ls_woe], df['tgt'])

0.7956699369111475

In [98]:
df["y_hat2"] = lr.predict(X = df[ls_woe])

In [99]:
df[['tgt', 'y_hat2']]

,tgt,y_hat2
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,1,0
9,0,0


In [100]:
df['y'] = (df['tgt'] == df['y_hat2']) * 1

In [101]:
df.y.sum()

28629

In [102]:
df.shape

(35981, 168)

In [103]:
roc_auc_score(y_true=df['tgt'], y_score=lr.predict(X=df[ls_woe]))

0.5240501960529923

#### Credit Scoring

In [ ]:
pdo = 10
base_score = 10
base_odds = 100
factor = pdo/np.log(2)
offset = base_score - (factor - np.log(base_odds))
m = len(ls_woe)

alpha = lr.intercept_
betas = lr.coef_[0]

In [ ]:
for feat, beta in zip(ls_woe, betas):
    df["P_" + feat[2:]] = df[feat].apply(lambda x:((beta*x + (alpha/m))*factor) + (offset/m)).astype(int)

In [ ]:
df.columns

In [ ]:
df["score"] = df[[x for x in df.columns if x.startswith("P_")]].sum(axis = 1)

In [ ]:
def plot_histogram(data, n_bins = 10):
    freq, bins = np.histogram(data, bins=n_bins)
    hist = pg.Histogram()
    hist.add('Wide bars', list(zip(freq, bins, bins[1:])))
    hist.render_to_png('histograma_scorecard')
    return hist

In [ ]:
plot_histogram(data = df["score"])

In [ ]:
df["score"].describe()

In [ ]:
ls_sc = [x[2:] for x in ls_woe] + [x for x in df.columns if x.startswith("P_")]

In [ ]:
ls_sc = [y[::-1] for y in sorted([x[::-1] for x in ls_sc])]

In [ ]:
ls_sc

In [ ]:
ls_sc_p = [x for x in ls_sc if x.startswith('P_')]

In [ ]:
ls_sc_sin_p = list(set(ls_sc) - set(ls_sc_p))

In [ ]:
for x in ls_sc_sin_p:
    display(df[[x, f'P_{x}']].drop_duplicates())

In [ ]:
df[["C_LDA_00", "P_C_LDA_00"]].drop_duplicates()

In [ ]:
# Regresión logística sin transformaciones
lr_orig = LogisticRegression(n_jobs=-1)
lr_orig.fit(X=df[ls_cont], y=df[tgt])
lr_orig.score(X=df[ls_cont], y=df[tgt])
lr_orig.predict(df[ls_cont]).sum()
roc_auc_score(y_true=df[tgt], y_score=lr_orig.predict(X=df[ls_cont]))